## import modules

In [1]:
import tensorflow as tf

## Hyper Parameter

In [2]:
EPOCHS = 10
NUM_WORDS = 10000

## Model Definition

In [3]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # 문장의 각 단어가 숫자로 코딩되어 있는데, 이는 클래스의 의미를 가지는 것으로 봐야 한다
        # (숫자가 비슷하다고 비슷한 의미를 갖거나 하지는 않음)
        self.emb = tf.keras.layers.Embedding(NUM_WORDS, 16)  # 길이 16의 feature가 32개 나열된 형태로 변경됨
        self.gru = tf.keras.layers.GRU(32)  # neuron의 개수 32
        self.dense = tf.keras.layers.Dense(1, activation='sigmoid')
        
    def call(self, x, training=None, mask=None):
        x = self.emb(x)
        x = self.gru(x)
        return self.dense(x)

## IMDB Dataset

입력: 문장(영화에 대한 평), 정답 : 0/1의 binary class (긍정/부정)

In [4]:
imdb = tf.keras.datasets.imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=NUM_WORDS)

print(x_train[:3])

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
D:\Installation\Anaconda3\envs\tf2.0\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


[list([1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32])
 list([1, 194, 1153, 194, 8255, 78, 228, 5, 6, 1463, 

D:\Installation\Anaconda3\envs\tf2.0\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


: 각 문장의 시작이 1로 되어 있음, 각 단어들은 숫자를 배정 받아서 표현됨

In [5]:
print(len(x_train[0]), len(x_train[1]))

218 189


: 각 문장의 길이가 다름 -> 앞부분 일부만 분석에 사용하자

In [6]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, value=0, padding='pre', maxlen=32)
                              # 32 길이로 자르고, 길이가 32가 안되는 경우에는 앞쪽에 0으로 padding 해줌
x_test = tf.keras.preprocessing.sequence.pad_sequences(x_test, value=0, padding='pre', maxlen=32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## Model Creation

In [7]:
model = MyModel()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Training

In [8]:
model.fit(train_ds, validation_data=test_ds, epochs=EPOCHS)

Epoch 1/10
782/782 [==============================] - 20s 22ms/step - loss: 0.5089 - accuracy: 0.7373 - val_loss: 0.4371 - val_accuracy: 0.7939
Epoch 2/10
782/782 [==============================] - 18s 23ms/step - loss: 0.3749 - accuracy: 0.8326 - val_loss: 0.4505 - val_accuracy: 0.7837
Epoch 3/10
782/782 [==============================] - 16s 20ms/step - loss: 0.3242 - accuracy: 0.8588 - val_loss: 0.4719 - val_accuracy: 0.7833
Epoch 4/10
782/782 [==============================] - 15s 19ms/step - loss: 0.2756 - accuracy: 0.8892 - val_loss: 0.5172 - val_accuracy: 0.7774
Epoch 5/10
782/782 [==============================] - 15s 20ms/step - loss: 0.2310 - accuracy: 0.9102 - val_loss: 0.5662 - val_accuracy: 0.7670
Epoch 6/10
782/782 [==============================] - 15s 20ms/step - loss: 0.1903 - accuracy: 0.9300 - val_loss: 0.6512 - val_accuracy: 0.7620
Epoch 7/10
782/782 [==============================] - 15s 19ms/step - loss: 0.1556 - accuracy: 0.9448 - val_loss: 0.7621 - val_accuracy:

-> val_loss가 오히려 계속 증가하고 있음.  
   Attention layer를 비롯한 추가적인 조치가 필요해 보임